# Домашнее задание № 4. Языковые модели

In [1]:
%%capture
!pip install razdel

In [2]:
import nltk
import numpy as np
import re

from collections import Counter
from nltk.tokenize import sent_tokenize
from scipy.sparse import lil_matrix, csc_matrix
from sklearn.model_selection import train_test_split
from string import punctuation
from razdel import tokenize as razdel_tokenize

In [3]:
nltk.download("punkt_tab")

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

## Задание 1 (8 баллов).

В семинаре для генерации мы использовали предположение маркова и считали, что слово зависит только от 1 предыдущего слова. Но ничто нам не мешает попробовать увеличить размер окна и учитывать два или даже три прошлых слова. Для них мы еще сможем собрать достаточно статистик и, логично предположить, что качество сгенерированного текста должно вырасти.

Попробуйте сделать языковую модель, которая будет учитывать два предыдущих слова при генерации текста.
Сгенерируйте несколько текстов (3-5) и расчитайте перплексию получившейся модели.
Можно использовать данные из семинара или любые другие (можно брать только часть текста, если считается слишком долго). Перплексию рассчитывайте на 10-50 отложенных предложениях (они не должны использоваться при сборе статистик).


Подсказки:  
    - нужно будет добавить еще один тэг \<start>  
    - можете использовать тот же подход с матрицей вероятностей, но по строкам хронить биграмы, а по колонкам униграммы
    - тексты должны быть очень похожи на нормальные (если у вас получается рандомная каша, вы что-то делаете не так)
    - у вас будут словари с индексами биграммов и униграммов, не перепутайте их при переводе индекса в слово - словарь биграммов будет больше словаря униграммов и все индексы из униграммного словаря будут формально подходить для словаря биграммов (не будет ошибки при id2bigram[unigram_id]), но маппинг при этом будет совершенно неправильным

Создадим корпус, токенизируем, поделим на train (для сбора статистик для марковской модели) и val (для подсчета переплексии).

In [4]:
news = open("lenta.txt").read()

In [5]:
def normalize(text):
    normalized_text = [
        word.text.strip(punctuation)
        for word in razdel_tokenize(text)
    ]
    normalized_text = [
        word.lower() for word in normalized_text
        if word and len(word) < 20
    ]
    return normalized_text

In [6]:
sents_news = [
    ["<start>"] * 2 + normalize(text) + ["<end>"]
    for text in sent_tokenize(news[:5000000])
]
sents_news_train, sents_news_val = train_test_split(
    sents_news, test_size=50, random_state=42
)

Создадим Counters для униграмм, биграмм и триграмм.

In [7]:
def ngrammer(tokens, n=2):
    ngrams = []
    for i in range(0, len(tokens)-n+1):
        ngrams.append(" ".join(tokens[i:i+n]))
    return ngrams

In [8]:
unigrams_news = Counter()
bigrams_news = Counter()
trigrams_news = Counter()

for sentence in sents_news_train:
    unigrams_news.update(sentence)
    bigrams_news.update(ngrammer(sentence))
    trigrams_news.update(ngrammer(sentence, n=3))

Создадим матрицу вероятностей, в которой по строкам хранятся биграммы, а по колонкам униграммы.

In [9]:
matrix_news = lil_matrix(
    (len(bigrams_news), len(unigrams_news))
)

id2word_news = list(unigrams_news)
word2id_news = {word:i for i, word in enumerate(id2word_news)}

id2bigram_news = list(bigrams_news)
bigram2id_news = {bigram:i for i, bigram in enumerate(id2bigram_news)}

for trigram in trigrams_news:
    bigram, word = trigram.rsplit(" ", 1)
    matrix_news[
        bigram2id_news[bigram],
        word2id_news[word]
    ] = (trigrams_news[trigram] / bigrams_news[bigram])

matrix_news = csc_matrix(matrix_news)

Перепишем функцию generate так, чтобы она генерировала следующее слово на основе любых n-грамм:

In [10]:
def generate(
    matrix, id2word, ngram2id, total_tokens=100, n=1
):
    start = " ".join(["<start>"] * n)
    text = []
    current_idx = ngram2id[start]
    prev_words = ["<start>"] * (n - 1)

    for i in range(total_tokens):
        chosen_idx = np.random.choice(
            matrix.shape[1],
            p=matrix[current_idx].toarray()[0]
        )
        chosen_word = id2word[chosen_idx]
        text.append(chosen_word)

        if chosen_word == "<end>":
            prev_words = ["<start>"] * (n - 1)
            chosen_word = "<start>"

        prev_words.append(chosen_word)
        current_idx = ngram2id[" ".join(prev_words)]
        prev_words = prev_words[1:]

    generation = re.sub(r"<end> ?", "\n", " ".join(text))
    return generation

### Генерации

In [11]:
for i in range(1, 6):
    print(f"Генерация {i}:")
    print(generate(matrix_news, id2word_news, bigram2id_news, n=2))
    print()

Генерация 1:
ранее аэрофлот повышал тарифы на перевозки грузов были увеличены на 10 
в письме начальника московского департамента образования любови кезиной новое положение позволит школьникам уклоняться от сдачи экзаменов и использовать его в так системе координат 
не надо кукарекать 
главным кандидатом на пост вице-мэра будет выдвигаться нынешний вице-мэр валерий шанцев сообщил что овр спс блока жириновского 
урон причиненный только сельскому хозяйству страны 
по ее словам уже удалось спасти невредимыми 
как сообщает итар-тасс со ссылкой на высокопоставленные источники знакомые с ходом расследования газета usa today что вашингтон не станет лишать ее свободы 
об этом по оценке

Генерация 2:
например на официальном заседании совета по подготовке компьютерных систем epa что проинформировали о них 
на это разрешение но завод кристалл не предпринял никаких действий по защите прав человека в страсбурге 
в апреле нынешнего года эти московские сми регулярно публикуют сообщения о намерении пр

### Перплексия

In [12]:
def compute_join_proba_markov_assumption(
    text, smaller_ngram_counts, ngram_counts,
    n=1, tokenized=True
):
    prob = 0
    if tokenized:
        tokenized_text = text
    else:
        tokenized_text = ["<start>"] * n + normalize(text) + ["<end>"]

    for ngram in ngrammer(tokenized_text, n=n+1):
        left_ngram, word = ngram.rsplit(" ", 1)
        if left_ngram in smaller_ngram_counts and ngram in ngram_counts:
            prob += np.log(
                ngram_counts[ngram] / smaller_ngram_counts[left_ngram]
            )
        else:
            prob += np.log(2e-5)
    return prob, len(tokenized_text) - n - 1

In [13]:
def perplexity(logp, N):
    return np.exp((-1/N) * logp)

In [14]:
def mean_perplexity(
    data_list, smaller_ngram_counts,
    ngram_counts, n=1, tokenized=True,
    print_logs=True
):
    ppl_sum = 0
    for sent in data_list:
        if print_logs:
            if tokenized:
                print(" ".join(sent[n:-1]))
            else:
                print(sent)

        current_ppl = perplexity(
            *compute_join_proba_markov_assumption(
                sent, smaller_ngram_counts, ngram_counts,
                n=n, tokenized=tokenized
            )
        )
        ppl_sum += current_ppl
        if print_logs:
            print("Перплексия:", current_ppl, "\n")
    return ppl_sum / len(data_list)

In [15]:
perplexity_val = mean_perplexity(
    sents_news_val, bigrams_news, trigrams_news, n=2
)

центральная избирательная комиссия утвердила перечень подлежащих опубликованию сведений о доходах и имуществе зарегистрированного кандидата на должность президента россии а также его супруги или супруга и детей
Перплексия: 804.866412443738 

е e причиной стала ошибка рабочих заливших в систему слишком много урановой смеси что привело к взрыву
Перплексия: 30058.841783487525 

идея построить мост через керченский пролив впервые пришла в голову немецкому министру вооружений альберту шпееру весной 1943 года
Перплексия: 24214.08364882088 

по словам представителей сирии переговоры действительно были отложены и стороны договариваются о новой дате очередного этапа переговорного процесса
Перплексия: 22558.37948352929 

тем не менее он не считает что это окончательная победа здравого смысла и справедливости в 2000 году во всех московских домах появятся консьержи и все подъезды будут оборудованы кнопкой тревожной сигнализации
Перплексия: 8681.802224985502 

стоимость перерегистрации домена также

Средняя перплексия модели на валидационном датасете:

In [16]:
perplexity_val

48560.1516120636

Средняя перплексия модели на фрагменте трейна (ожидаемо меньше):

In [17]:
mean_perplexity(
    sents_news_train[:1000], bigrams_news, trigrams_news, n=2,
    print_logs=False
)

37.09537163547413

## Задание № 2* (2 балла).

Измените функцию generate_with_beam_search так, чтобы она работала с моделью, которая учитывает два предыдущих слова.
Сравните получаемый результат с первым заданием.
Также попробуйте начинать генерацию не с нуля (подавая \<start> \<start>), а с какого-то промпта. Но помните, что учитываться будут только два последних слова, так что не делайте длинные промпты.

In [18]:
class Beam:
    def __init__(self, sequence: list, score: float):
        self.sequence: list = sequence
        self.score: float = score

In [19]:
def generate_with_beam_search(
    matrix, id2word, ngram2id,
    total_tokens=100, max_beams=5,
    n=1, prompt=""
):
    # изначально у нас один луч с заданным началом (start по дефолту)
    initial_node = Beam(
        sequence=["<start>"] * n + normalize(prompt),
        score=np.log1p(0)
    )
    beams = [initial_node]

    for i in range(total_tokens):
        # делаем total_tokens шагов генерации
        new_beams = []
        # на каждом шаге продолжаем каждый из имеющихся лучей
        for beam in beams:
            # лучи которые уже закончены не продолжаем (но и не удаляем)
            if beam.sequence[-1] == "<end>":
                new_beams.append(beam)
                continue

            # наша языковая модель предсказывает на основе предыдущей n-граммы
            # достанем его из beam.sequence
            last_id = ngram2id[" ".join(beam.sequence[-n:])]

            # посмотрим вероятности продолжений для предыдущей n-граммы
            probas = matrix[last_id].toarray()[0]

            # возьмем топ самых вероятных продолжений
            top_idxs = probas.argsort()[:-(max_beams+1):-1]
            for top_id in top_idxs:
                # иногда вероятности будут нулевые, такое не добавляем
                if not probas[top_id]:
                    break

                # создадим новый луч на основе текущего и варианта продолжения
                new_sequence = beam.sequence + [id2word[top_id]]
                # скор каждого луча это произведение вероятностей (или сумма логарифмов)
                new_score = (beam.score + np.log1p(probas[top_id])) / len(new_sequence)
                new_beam = Beam(sequence=new_sequence, score=new_score)
                new_beams.append(new_beam)
        # отсортируем лучи по скору и возьмем только топ max_beams
        beams = sorted(new_beams, key=lambda x: x.score, reverse=True)[:max_beams]

    sorted_sequences = sorted(beams, key=lambda x: x.score, reverse=True)
    sorted_sequences = [" ".join(beam.sequence[n:-1]) for beam in sorted_sequences]
    return sorted_sequences

Генерация с нуля:

In [20]:
generate_with_beam_search(
    matrix_news, id2word_news,
    bigram2id_news, max_beams=10, n=2,
)

['как сообщили риа новости',
 'как сообщает риа новости',
 'об этом риа новости',
 'об этом сообщает риа новости',
 'об этом сообщает агентство риа новости',
 'об этом сообщает риа новости со ссылкой на пресс-центр мвд участники совещания заслушали руководителей подразделений на приоритетных направлениях работы защите экономики от криминального влияния ”',
 'об этом сообщает риа новости со ссылкой на пресс-центр объединенной группировки войск на северном кавказе',
 'об этом сообщает риа новости со ссылкой на пресс-центр мвд участники совещания заслушали руководителей подразделений на территории чечни',
 'об этом сообщает риа новости со ссылкой на пресс-центр объединенной группировки федеральных войск на северном кавказе',
 'об этом сообщает риа новости со ссылкой на пресс-центр мвд участники совещания заслушали руководителей подразделений на приоритетных направлениях работы защите экономики от криминального влияния противодействии наркомании и борьбе с терроризмом']

Генерации с промптами:

In [21]:
generate_with_beam_search(
    matrix_news, id2word_news,
    bigram2id_news, max_beams=10, n=2,
    prompt="в санкт-петербурге"
)

['в санкт-петербурге а вернее в петербургском интернете',
 'в санкт-петербурге было совершено покушение',
 'в санкт-петербурге а вернее в петербургском отделении национального института прессы',
 'в санкт-петербурге а вернее в петербургском отделении яблока считают поджог',
 'в санкт-петербурге на подготовку и переподготовку кадров для новых предприятий электронной промышленности зеленограда',
 'в санкт-петербурге избит кандидат в президенты россии',
 'в санкт-петербурге на подготовку и переподготовку кадров для новых солдат сценарий учения будет основываться на подлинных документах',
 'в санкт-петербурге на подготовку и переподготовку кадров для новых солдат сценарий учения будет основываться на американской бирже nasdaq',
 'в санкт-петербурге на подготовку и переподготовку кадров для новых предприятий электронной промышленности',
 'в санкт-петербурге на подготовку и переподготовку кадров для новых солдат сценарий учения будет основываться на американской бирже']

Пример из первого задания:

```по словам тулеева в результате были задержана группа лиц которые могут возникнуть из-за отказа тормозов```

In [22]:
generate_with_beam_search(
    matrix_news, id2word_news,
    bigram2id_news, max_beams=10, n=2,
    prompt="по словам"
)

['по словам начальника пресс-службы гувд владимир зубков отказался прокомментировать происшествие',
 'по словам начальника пресс-службы гувд александра ростовцева сообщил иную версию происшествия',
 'по словам вешнякова этот вопрос',
 'по словам пресс-секретаря премьера владимир путин',
 'по словам пресс-секретаря премьера многие пункты плана путин счел достойными внимания однако им было получено вполне конкретное телефонное распоряжение кавказцев не регистрировать',
 'по словам начальника отдела по борьбе с коррупцией',
 'по словам начальника отдела по борьбе с терроризмом',
 'по словам начальника отдела по борьбе с организованной преступностью',
 'по словам пресс-секретаря премьера многие пункты плана путин счел достойными внимания однако им была раскритикована мысль о том что в ходе встречи с президентом сша биллом клинтоном',
 'по словам пресс-секретаря премьера многие пункты плана путин счел достойными внимания однако им была раскритикована мысль о том что в ходе встречи с президе

Пример из первого задания:

```на заседание в четверг этих людей известен как основатель республиканского национального банка нью-йорка```

In [23]:
generate_with_beam_search(
    matrix_news, id2word_news,
    bigram2id_news, max_beams=10, n=2,
    prompt="на заседание"
)

['на заседание самого доренко оказались безуспешными',
 'на заседание координационного совета движения',
 'на заседание думы сначала не хотели пропускать диссидентов и забаррикадировались внутри храма закрыв входы металлическими контейнерами и колючей проволокой',
 'на заседание координационного совета движения города россии снг и балтии',
 'на заседание представителей белорусской оппозиции участников митинга к спокойствию и правопорядка',
 'на заседание представителей белорусской оппозиции участников митинга к спокойствию и правопорядка он попросил передать на борт продукты питания палатки теплые вещи медикаменты продуктыпитания',
 'на заседание представителей белорусской оппозиции участников митинга к спокойствию и правопорядка он попросил не задавать вопросов по деталям этого дела',
 'на заседание представителей белорусской оппозиции участников митинга к спокойствию и правопорядка он попросил не задавать вопросов по урегулированию ситуации в чечне',
 'на заседание представителей бел

Пример из первого задания:

```мы путешественники и нам хочется узнать как можно быстрее вызволить их из великобритании```

In [24]:
generate_with_beam_search(
    matrix_news, id2word_news,
    bigram2id_news, max_beams=10, n=2,
    prompt="мы путешественники"
)

['мы путешественники и нам этого достаточно',
 'мы путешественники и нам показалось что кто-то вырезает телефонный кабель',
 'мы путешественники и нам видимо работа достанется добавил он',
 'мы путешественники и нам хочется узнать как можно больше о странах которые пересекаем',
 'мы путешественники и нам всегда было интересно попробовать себя и москвичей от гастролеров от бандитов освобожденных участков города',
 'мы путешественники и нам показалось что листовки могут появиться в штабе объединенной группировки войск на северном кавказе',
 'мы путешественники и нам хочется узнать как можно больше снова наворовать под прикрытием восстановления нефтяной отрасли республики беспокоится глава госсовета',
 'мы путешественники и нам всегда было интересно попробовать себя и москвичей от гастролеров от бандитов покинуть их населенные пункты',
 'мы путешественники и нам хочется узнать как можно больше снова наворовать под прикрытием восстановления нефтяной отрасли республики беспокоится глава гос

По сравнению с первым заданием наиболее вероятные генерации beam search
- более короткие (токен eos имеет высокую вероятность)
- более "предсказуемые" (по определению, потому что в первом задании мы семплировали по всему словарю)